In [2]:
%load_ext autoreload 
%autoreload 2

!pip install --upgrade boto3
!pip install --upgrade sagemaker
!pip install python_docx
!pip install langchain
!pip install pypdf
!pip install docx2txt
!pip install unstructured

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 24.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.157
    Uninstalling botocore-1.29.157:
      Successfully uninstalled botocore-1.29.157
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.157
    Uninstalling boto3-1.26.157:
      Successfully uninstalled boto3-1.26.157
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.157 requires botocore==1.29.157, but you have botocore 1.31.26 which is incompatible.
sagemaker 2.167.0 requires PyYAML==6.0, but you have pyyaml 5.4.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, ht

  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.167.0
    Uninstalling sagemaker-2.167.0:
      Successfully uninstalled sagemaker-2.167.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.157 requires botocore==1.29.157, but you have botocore 1.31.26 which is incompatible.
awscli 1.27.157 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.
sparkmagic 0.20.5 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
sparkmagic 0.20.5 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.0.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.0/271.0 kB 911.5 kB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3957 sha256=e096b90436b3f6422294061949a8262bce9dc0e141fa2d6bb03ebe95644bc081
  Stored in directory: /home/ec2-user/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel 

In [3]:
import os
import shutil
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredPowerPointLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import NLTKTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from chinese_text_splitter import ChineseTextSplitter
import json
from typing import Dict, List, Tuple
from tqdm import tqdm
from datetime import datetime

import boto3
import nltk
import pandas as pd
import logging

import sagemaker
from sagemaker.huggingface import HuggingFaceModel
import requests
import re

#nltk.download('punkt')
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

## Hyperparameter

In [4]:
# The name of index
sm_client = boto3.client('secretsmanager')
index_name = sm_client.get_secret_value(SecretId='opensearch-index-name')['SecretString']
data= json.loads(index_name)
index_name = data.get('index')
print('pre-defined index name in deployment/cdk.json-->',index_name)
index_name = '08152200'

# Language, 'chinese' or 'english'
language = 'chinese'

# The name of embbeding model endpoint, usually you can keep it as default
eb_endpoint = 'huggingface-inference-eb'

# Ebbeding vector dimension, usually you can keep it as default
v_dimension = 768

# Docs file folder to be processed and ingested
folder_path = 'docs/'
print('Please put data in this folder-->',folder_path)

# Paragraph size / Chunck size
chunck_size = 200

# The imported data of the same index_name, usually you can keep it as 0 if you are creating a new index
before_import = 0

pre-defined index name in deployment/cdk.json--> smart_search_qa_test
Please put data in this folder--> docs/


In [9]:
#test
names = os.listdir(folder_path)
# before_import = 0
failed_files = []
#for j in range(len(names)):
#name = names[j]
name = names[1]
#     if os.path.splitext(name)[1] not in ['.doc','.docx']:continue
print(name)
loader = PyPDFLoader(os.path.join(folder_path, name))
#print("loader.load()")
#print(loader.load())
docs = loader.load_and_split(ChineseTextSplitter())
print(docs)

CROWN_2012.pdf
[Document(page_content='$\x12\n.&+\n4\n,\n-\n6%\n3\x13\n\x14\x15\x16\x17\x18\x19\x1a\x11', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 0}), Document(page_content='目录\n11驾驶前 调节和操作门锁、后视镜和转向柱等装备。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n2驾驶过程中 驾驶、停车和安全驾驶信息。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n3车内装备空调和音响系统及其他用于营造驾驶舒适感的车内\n装备。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n4保养和维护 车辆的清洁和保护、自行保养及保养信息。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n5出现故障时车辆需要拖拽、轮胎泄气或发生事故时的应对措施。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n6车辆规格 车辆详细信息。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='\n索引 列举本手册所包含的信息。', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 2}), Document(page_content='目录 索引\n', metadata={'source': 'docs/CROWN_2012.pdf', 'page': 3}),

In [6]:
#test
import re

text = re.sub(r"\n{3,}", "\n", "111-2. 打开、关闭和锁止车门与行李厢\n1驾驶前\n86_GC■无线遥控装置的认证证书\n")
text = re.sub('\s', ' ', text)
text = text.replace("\n\n", "。")
pre_txt = ""
print(text)
sent_sep_pattern = re.compile('([■●﹒﹔﹖﹗．。！？]|\d{1,3}-\d{1,3}\.|$)') 
pre_sep_pattern = re.compile('([■●]|\d{1,3}-\d{1,3}\.)') 
stop_pattern = re.compile('.*→P\..*')
sent_list = []
for ele in sent_sep_pattern.split(text):
    print(ele)
    searchObj = re.search( "("+str(int("12")-1)+'){0,1}(\d{1,3}-\d{1,3}\.)', ele)
    if searchObj: 
        print("page:",searchObj.group(2))
    if stop_pattern.match(ele) :
        pre_txt = ""
    elif pre_sep_pattern.match(ele) :
        pre_txt = ele
    elif sent_sep_pattern.match(ele) and sent_list:
        sent_list[-1] += ele
    elif ele:
        sent_list.append(pre_txt+ele)
        pre_txt = ""
print(sent_list)

111-2. 打开、关闭和锁止车门与行李厢 1驾驶前 86_GC■无线遥控装置的认证证书 

111-2.
page: 1-2.
 打开、关闭和锁止车门与行李厢 1驾驶前 86_GC
■
无线遥控装置的认证证书 


['111-2. 打开、关闭和锁止车门与行李厢 1驾驶前 86_GC', '■无线遥控装置的认证证书 ']


In [7]:
hfp = sagemaker.huggingface.model.HuggingFacePredictor(eb_endpoint)

#===================Function Definition=================

def load_file(filepath,language):
    
    if filepath.lower().endswith(".pdf"):
        loader = PyPDFLoader(filepath)
    elif filepath.lower().endswith(".docx"):
        loader = Docx2txtLoader(filepath)
    elif filepath.lower().endswith(".pptx"):
        loader = UnstructuredPowerPointLoader(filepath)
    elif filepath.lower().endswith(".csv"):
        loader = CSVLoader(filepath)
    elif filepath.lower().endswith(".xlsx"):
        loader = UnstructuredExcelLoader(filepath)
    elif filepath.lower().endswith(".txt"):
        loader = TextLoader(filepath)
    else:
        loader = TextLoader(filepath)
    
    if language == "chinese":
        textsplitter = ChineseTextSplitter()
    elif language == "english":
        textsplitter = NLTKTextSplitter(chunk_size=chunck_size, chunk_overlap=10)

    docs = loader.load_and_split(textsplitter)
    return docs


def get_title(path):
    try:
        title = os.path.split(os.path.splitext(path)[0])[1].replace('——', '-').split('-')[1]
    except:
        title = os.path.split(os.path.splitext(path)[0])[1]
    return title

def read_doc(path, chunck_size = chunck_size):
    doc = load_file(path, language)
    title = get_title(path)
    titles = []
    paragraphs = []
    sentences = []
    pages = []
    para = ''
    con = 0
    para_split = ""
    for d in doc:
#         print('*********')
        titles.append(title)
        sentences.append(d.page_content)
#        print("sentences:",d.page_content)
        pages.append(d.metadata['page'])
        searchObj = re.search( "("+str(d.metadata['page']-1)+'){0,1}(\d{1,3}-\d{1,3}\.)', d.page_content)
        if searchObj: 
            if para_split != searchObj.group(2):
                paragraphs += [para for _ in range(con)]
                para = ''
                con = 0
                para_split = searchObj.group(2)
#                print("paragraphs:",searchObj.group(2))
#        if len(para) >= chunck_size:
#            paragraphs += [para for _ in range(con)]
#            para = ''
#            con = 0
        con += 1
        para += d.page_content
    paragraphs += [para for _ in range(con)]
    print(len(titles), len(sentences),len(paragraphs))
    df = pd.DataFrame({'title':titles, 'paragraph':paragraphs, 'sentence':sentences,'page':pages})
    return df

def get_vector(q):
    try:
        vector = hfp.predict({'inputs':[q]})[0][0][0]
        return vector
    except:
        return [-1000 for _ in range(v_dimension)]
    return hfp.predict({'inputs':[q]})[0][0][0]

def embbeding(df):
#    df['title_vector'] = ''
    df['sentence_vector'] = ''
    title_vector = str(get_vector(df.iloc[0, 0]))
    for i in range(len(df)):
#         df.iloc[i, 5] = title_vector
        df.iloc[i, 4] = str(get_vector(df.iloc[i, 2]))
        print('\r embbeding %i out of %i finished'%(i, len(df)), end='')
    return df

# ==============OpenSearch Related=====================
# retrieve secret manager value by key using boto3
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
region = boto3.Session().region_name # e.g. cn-north-1
# sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')
# service = 'es'
# credentials = boto3.Session().get_credentials()
awsauth = (username, password)
url = host+'_bulk'
headers = { "Content-Type": "application/json" }

payloads = {
"settings": { "index": {
"knn": True,
"knn.algo_param.ef_search": 100 }
}, "mappings": {
"properties": { 
  "title_vector": {
"type": "knn_vector", "dimension": v_dimension, "method": {
"name": "hnsw", "space_type": "l2", "engine": "nmslib", "parameters": {
"ef_construction": 256,
"m": 128 }
} },
"sentence_vector": {
"type": "knn_vector", "dimension": v_dimension, "method": {
"name": "hnsw", "space_type": "l2", "engine": "nmslib", "parameters": {
"ef_construction": 256,
"m": 128 }
} },
"title": { "type": "text"}, 
"sentence": {"type": "text" }, 
"paragraph": {"type": "text" }, 
"sentence_id": {"type": "text" }, 
"paragraph_id": {"type": "text" }
} }
}

# Create Index
#r = requests.put(host+index_name, auth=awsauth, headers=headers, json=payloads)

def import_data(df, id_start=0, before_import=0, idx_name=""):
    payloads = ''
    for i in range(id_start, len(df)+id_start):
        first = json.dumps({ "index": { "_index": idx_name, "_id": str(i+before_import) } }, ensure_ascii=False) + "\n"
        second = json.dumps({"title": str(df.iloc[i-id_start, 0]), 
                     "paragraph": str(df.iloc[i-id_start, 1]), 
                     "sentence": str(df.iloc[i-id_start, 2]), 
                     "sentence_vector": json.loads(df.iloc[i-id_start, 4]),
                     "page": df.iloc[i-id_start, 3]},
                   ensure_ascii=False) + "\n"
        payloads += first + second
    # print(payloads)
    r = requests.post(url, auth=awsauth, headers=headers, data=payloads.encode()) # requests.get, post, and delete have similar syntax
#     print(r.text)

#==============Main Preprocess Data and Import===============

slice = 10
names = os.listdir(folder_path)
# before_import = 0
failed_files = []
for j in range(len(names)):
    name = names[j]
#     if os.path.splitext(name)[1] not in ['.doc','.docx']:continue
    print('filename-->',name)
    # Create Index
    r = requests.put(host+name.split("_")[0].lower()+index_name, auth=awsauth, headers=headers, json=payloads)
    print('index_name-->',name.split("_")[0].lower()+index_name)
    if os.path.isdir(name):continue
    try:
        df = read_doc(os.path.join(folder_path, name))
        df = embbeding(df)
        for i in range(len(df)//slice+1):
            import_data(df[slice*i:slice*(i+1)], slice*i, before_import,name.split("_")[0].lower()+index_name)
            print('\r import %i out of %i finished'%(i, len(df)//slice+1), end='')
        before_import += len(df)
        print(' file %i out of %i finished'%(j, len(names)//slice+1))
    except Exception as ex:
        # traceback.print_exc(file=sys.stdout)
        failed_files.append(name)
        print(f"=================Exception================={ex}")

filename--> .ipynb_checkpoints
index_name--> .ipynb08152200
filename--> CROWN_2012.pdf
index_name--> crown08152200
4300 4300 4300
 import 430 out of 431 finished file 1 out of 1 finished
